# Day 19

In [1]:
with open('../inputs/adventofcode.com_2022_day_19_input.txt', 'r') as f:
    data = f.read().splitlines()

print(f'The file contains {len(data)} Blueprints.')

The file contains 30 Blueprints.


In [2]:
import numpy as np

class Node():
    def __init__(self, minute, ore_robots, clay_robots=0, obsidian_robots=0, geode_robots=0, 
                 ore_units=0, clay_units=0, obsidian_units=0, geode_units=0, robot_cost={}, new_robot=None):
        self.minute = minute
        
        self.ore_robots = ore_robots
        self.clay_robots = clay_robots
        self.obsidian_robots = obsidian_robots
        self.geode_robots = geode_robots

        # Mining 
        # Every new minute we have the previous number of units plus one for each robot that is mining.
        self.ore_units = ore_units
        self.clay_units = clay_units
        self.obsidian_units = obsidian_units
        self.geode_units = geode_units 

        self.robot_cost = robot_cost
        
        if new_robot == 'ore':
            self.ore_robots += 1
        elif new_robot == 'clay':
            self.clay_robots += 1
        elif new_robot == 'obsidian':
            self.obsidian_robots += 1
        elif new_robot == 'geode':
            self.geode_robots += 1


    def __repr__(self):
        return f'At minute {self.minute} we have {self.ore_units} ore, {self.clay_units} clay, {self.obsidian_units} obsidian and {self.geode_units} geodes. Robots: {self.ore_robots} ore, {self.clay_robots} clay, {self.obsidian_robots} obsidian, {self.geode_robots} geode.'

    
    def build_robot(self, robot_type):
        # Compute minutes needed to have enough ore to build a new clay robot.
        if robot_type == 'ore':
            needed_minutes = max(1, np.ceil((self.robot_cost[robot_type]['ore'] - self.ore_units) / self.ore_robots)+1)
        elif robot_type == 'clay':
            needed_minutes = max(1, np.ceil((self.robot_cost[robot_type]['ore'] - self.ore_units) / self.ore_robots)+1)
        elif robot_type == 'obsidian':
            needed_minutes_ore =        max(1, np.ceil((self.robot_cost[robot_type]['ore'] - self.ore_units) / self.ore_robots)+1)
            needed_minutes_clay =       max(1, np.ceil((self.robot_cost[robot_type]['clay'] - self.clay_units) / self.clay_robots)+1)
            needed_minutes = max(needed_minutes_ore, needed_minutes_clay)
        elif robot_type == 'geode':
            needed_minutes_ore =        max(1, np.ceil((self.robot_cost[robot_type]['ore'] - self.ore_units) / self.ore_robots)+1)
            needed_minutes_obsidian =   max(1, np.ceil((self.robot_cost[robot_type]['obsidian'] - self.obsidian_units) / self.obsidian_robots)+1)
            needed_minutes = max(needed_minutes_ore, needed_minutes_obsidian)
        return __class__(self.minute + needed_minutes, self.ore_robots, self.clay_robots, self.obsidian_robots, self.geode_robots,
                        self.ore_units + needed_minutes*self.ore_robots - self.robot_cost[robot_type]['ore'], 
                        self.clay_units + needed_minutes*self.clay_robots - self.robot_cost[robot_type]['clay'], 
                        self.obsidian_units + needed_minutes*self.obsidian_robots - self.robot_cost[robot_type]['obsidian'], 
                        self.geode_units + needed_minutes*self.geode_robots, 
                        robot_cost=self.robot_cost, new_robot=robot_type
                        )


    def next_steps(self):
        # To build ore and clay robots we need ore (that is produced by ore robots)
        if self.ore_robots:
            if self.ore_robots < self.robot_cost['max']['ore'] and \
                self.ore_units + (max_time -1 - self.minute) * (self.ore_robots) < self.robot_cost['max']['ore'] * (max_time -1 - self.minute):
                yield self.build_robot('ore')
            if self.clay_robots < self.robot_cost['max']['clay'] and \
                self.clay_units + (max_time -1 - self.minute) * (self.clay_robots) < self.robot_cost['max']['clay'] * (max_time -1 - self.minute):
                yield self.build_robot('clay')

        # To build obsidian robots we need ore and clay.
        if self.ore_robots and self.clay_robots:
            if self.obsidian_robots < self.robot_cost['max']['obsidian'] and \
                self.obsidian_units + (max_time -1 - self.minute) * (self.obsidian_robots) < self.robot_cost['max']['obsidian'] * (max_time -1 - self.minute):
            # if self.obsidian_units + (max_time -1 - self.minute) * (self.obsidian_robots) < self.robot_cost['geode']['obsidian'] * (max_time -1 - self.minute):
                # We can build an obsidian robot when we have enough ore and clay.
                # yield self.build_obsidian_robot()
                yield self.build_robot('obsidian')

        # To build geode robots we need ore and obsidian.
        if self.ore_robots and self.obsidian_robots:
            # We can build a geode robot when we have enough ore and obsidian.
            # yield self.build_geode_robot()
            yield self.build_robot('geode')

In [3]:
from collections import deque
import re

pattern = r'Blueprint \d+: Each ore robot costs (\d+) ore\. Each clay robot costs (\d+) ore\. Each obsidian robot costs (\d+) ore and (\d+) clay\. Each geode robot costs (\d+) ore and (\d+) obsidian\.'

def max_geodes(bp):

    matches = re.search(pattern, bp).groups()

    robot_cost = {'ore': {'ore': int(matches[0]), 'clay': 0, 'obsidian': 0},
                  'clay': {'ore': int(matches[1]), 'clay': 0, 'obsidian': 0},
                  'obsidian': {'ore': int(matches[2]), 'clay': int(matches[3]), 'obsidian': 0},
                  'geode': {'ore': int(matches[4]), 'clay': 0, 'obsidian': int(matches[5])}
                 }
    
    robot_cost['max'] = {material: max([robot_cost[m][material] for m in robot_cost.keys()]) for material in ['ore', 'clay', 'obsidian']}

    queue = deque()
    queue.append(Node(minute=1, ore_robots=1, clay_robots=0, obsidian_robots=0, geode_robots=0, 
                    ore_units=1, clay_units=0, obsidian_units=0, geode_units=0, robot_cost=robot_cost))

    max_geodes = []
    max_num_geodes = 0

    while queue:
        current_node = queue.popleft()

        for n in current_node.next_steps():

            if n.minute < max_time:
                if n.geode_units + n.geode_robots * (max_time - n.minute) + sum([i for i in range(max_time - int(n.minute))]) <= max_num_geodes:
                    continue 
                queue.append(n)
                max_num_geodes = max(max_num_geodes, n.geode_units)

            else:
                num_geodes = current_node.geode_units + current_node.geode_robots * (max_time - current_node.minute)
                max_geodes.append(num_geodes)
                max_num_geodes = max(max_num_geodes, num_geodes)
                
    return max(max_geodes)



## Puzzle 1

In [4]:
max_time = 24
# Compute the quality level for each blueprint, and add them up.
quality_level = 0
for i, bp in enumerate(data):
    quality_level += max_geodes(bp) * (i+1)

print(f'The quality level is {quality_level:g}.')

The quality level is 1115.


## Puzzle 2

In [5]:
max_time = 32

best_level = []
for i, bp in enumerate(data[:3]):
    best_level.append(max_geodes(bp))

print(f'{best_level=}')
print(f'The answer is {np.product(best_level):g}.')

best_level=[16.0, 54.0, 29.0]
The answer is 25056.
